<a href="https://colab.research.google.com/github/otanet/DLB2023_matsuo-lab_202304-06/blob/main/sub_lecture03_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第3回講義 宿題

## 課題

今Lessonで学んだことを元に，MNISTのファッション版 (Fashion MNIST，クラス数10) を多層パーセプトロンによって分類してみましょう．

Fashion MNISTの詳細については以下のリンクを参考にしてください．

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値

Accuracy 85%

### ルール

- 訓練データはx_train， t_train，テストデータはx_testで与えられます．
- 予測ラベルは one_hot表現ではなく0~9のクラスラベル で表してください．
- **下のセルで指定されているx_train，t_train以外の学習データは使わないでください．**
- **多層パーセプトロンのアルゴリズム部分は第3回の演習を参考に，NumPyのみで実装してください．** (sklearnやtensorflowなどは使用しないでください)．
    - データの前処理部分でsklearnの関数を使う (例えば sklearn.model_selection.train_test_split) のは問題ありません．

### 提出方法

- 2つのファイルを提出していただきます．
  - テストデータ (x_test) に対する予測ラベルをcsvファイル (ファイル名: submission_pred.csv) で提出してください．
  - それに対応するpythonのコードをsubmission_code.pyとして提出してください (%%writefileコマンドなどを利用してください)．

### 評価方法

- 予測ラベルのt_testに対する精度 (Accuracy) で評価します．
- 提出後即時採点を行い，Leader Boardが更新されます．
- 締切後の点数を最終的な評価とします．

In [ ]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

### データの読み込み

- この部分は修正しないでください

In [14]:
import os
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import inspect


In [19]:
#学習データ
x_train = np.load('/content/drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture02/data/x_train.npy')
# x_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture03/data/x_train.npy')
t_train = np.load('/content/drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture02/data/y_train.npy')
    
#テストデータ
x_test = np.load('/content/drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture02/data/x_test.npy')


In [20]:
# データの前処理（正規化， one-hot encoding)
x_train, x_test = x_train / 255., x_test / 255.
x_train, x_test = x_train.reshape(x_train.shape[0], -1), x_test.reshape(x_test.shape[0], -1)
t_train = np.eye(N=10)[t_train.astype("int32").flatten()]

### 多層パーセプトロンの実装

In [21]:
# データの分割
x_train, x_val, t_train, t_val =\
    train_test_split(x_train, t_train, test_size=10000)

In [22]:
def np_log(x):
    return np.log(np.clip(x, 1e-10, 1e+10))


def create_batch(data, batch_size):
    """
    :param data: np.ndarray，入力データ
    :param batch_size: int，バッチサイズ
    """
    num_batches, mod = divmod(data.shape[0], batch_size)
    batched_data = np.split(data[: batch_size * num_batches], num_batches)
    if mod:
        batched_data.append(data[batch_size * num_batches:])

    return batched_data

In [29]:
# シード値を変えることで何が起きるかも確かめてみてください．
rng = np.random.RandomState(1234)
random_state = 42


# 発展: 今回の講義で扱っていない活性化関数について調べ，実装してみましょう
def relu(x):
    return np.maximum(0, x)


def deriv_relu(x):
    return np.where(x <= 0, 0, 1)


def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)


def deriv_softmax(x):
    p = softmax(x)
    return p * (1 - p)


def crossentropy_loss(t, y):
    return -np.sum(t * np.log(y)) / t.shape[0]


class Dense:
    def __init__(self, in_dim, out_dim, function, deriv_function):
        self.W = rng.normal(scale=0.1, size=(in_dim, out_dim))
        self.b = np.zeros((out_dim,))
        self.function = function
        self.deriv_function = deriv_function

    def __call__(self, x):
        return self.function(x @ self.W + self.b)

    def bprop(self, x, delta):
        delta = self.deriv_function(self(x)) * delta
        grad_W = x.T @ delta
        grad_b = np.sum(delta, axis=0)
        delta = delta @ self.W.T
        return delta, grad_W, grad_b


class Model:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def train(self, x, t, lr):
        batched_x = create_batch(x, batch_size)
        batched_t = create_batch(t, batch_size)

        for i in range(len(batched_x)):
            x_batch = batched_x[i]
            t_batch = batched_t[i]
            y = self(x_batch)
            delta = (y - t_batch) / batch_size
            for layer in reversed(self.layers):
                delta, grad_W, grad_b = layer.bprop(layer(x_batch), delta)
                layer.W -= lr * grad_W
                layer.b -= lr * grad_b

    def test(self, x):
        y = self(x)
        return np.argmax(y, axis=1)


lr = 0.1
n_epochs = 100
batch_size = 100

mlp = Model([
    Dense(784, 200, relu, deriv_relu),
    Dense(200, 10, softmax, deriv_softmax)
])


### モデルの学習

In [35]:
def train_model(mlp, x_train, t_train, x_val, t_val, n_epochs=10):
    for epoch in range(n_epochs):
        losses_train = []
        losses_valid = []
        train_num = 0
        train_true_num = 0
        valid_num = 0
        valid_true_num = 0

        x_train, t_train = shuffle(x_train, t_train)
        x_train_batches, t_train_batches = create_batch(x_train, batch_size), create_batch(t_train, batch_size)

        x_val, t_val = shuffle(x_val, t_val)
        x_val_batches, t_val_batches = create_batch(x_val, batch_size), create_batch(t_val, batch_size)

        # モデルの訓練
        for x, t in zip(x_train_batches, t_train_batches):
            # 順伝播
            y = mlp.forward(x) # WRITE ME

            # 損失の計算
            loss = mlp.compute_loss(y, t) # WRITE ME     
            losses_train.append(loss.tolist())

            # パラメータの更新
            mlp.backward(loss)
            mlp.update_params() # WRITE ME

            # 精度を計算
            acc = accuracy_score(t.argmax(axis=1), y.argmax(axis=1), normalize=False)
            train_num += x.shape[0]
            train_true_num += acc

        # モデルの評価
        for x, t in zip(x_val_batches, t_val_batches):
            # 順伝播
            y = mlp.forward(x) # WRITE ME

            # 損失の計算
            loss = mlp.compute_loss(y, t) # WRITE ME
            losses_valid.append(loss.tolist())

            acc = accuracy_score(t.argmax(axis=1), y.argmax(axis=1), normalize=False)
            valid_num += x.shape[0]
            valid_true_num += acc.sum().item()

        print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
            epoch,
            np.mean(losses_train),
            train_true_num/train_num,
            np.mean(losses_valid),
            valid_true_num/valid_num
        ))


In [41]:
t_pred = []
for x in x_test:
    # 順伝播
    x = x[np.newaxis, :]
    y = mlp(x)

    # モデルの出力を予測値のスカラーに変換
    pred = y.argmax(1).tolist()

    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv('/submission_pred.csv', header=True, index_label='id')